In [10]:
import os
import re
import pandas as pd
from pybaseball import statcast_batter, statcast_pitcher, playerid_lookup, pitching_stats_range, batting_stats_range, schedule_and_record, team_game_logs, batting_stats
import datetime 
import statsapi
from statsapi import player_stat_data
import pprint

In [44]:
# Define the list of player IDs
player_ids = ['682928']  # Add all the player IDs you need

# Define the years you want to process
years = [2021, 2022, 2023, 2024]

# Define the folder containing the XLS files
folder_path = 'batter_historical'

# Loop through each player ID
for player_id in player_ids:
    # Create an empty list to store DataFrames
    player_data = []
    
    # Loop through each year
    for year in years:
        # Construct the file name
        file_name = f"{player_id}_{str(year)[-2:]}.xlsx"
        file_path = os.path.join(folder_path, file_name)
        
        # Check if the file exists
        if os.path.exists(file_path):
            try:
                # Try reading the file as an Excel file
                df = pd.read_excel(file_path)
            except ValueError:
                try:
                    # If reading as Excel fails, try reading as HTML
                    df = pd.read_html(file_path)[0]
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
                    continue
            # Append the DataFrame to the list
            player_data.append(df)
        else:
            print(f"File not found: {file_path}")
    
    # Concatenate all DataFrames in the list
    if player_data:
        concatenated_df = pd.concat(player_data)
        
        # Save the concatenated DataFrame to a new file
        output_file = os.path.join(folder_path, f"{player_id}.xlsx")
        concatenated_df.to_excel(output_file, index=False)
        print(f"Concatenated file saved for player {player_id}: {output_file}")
    else:
        print(f"No data found for player {player_id}")

print("All players processed.")

File not found: batter_historical\682928_21.xlsx
Concatenated file saved for player 682928: batter_historical\682928.xlsx
All players processed.


In [56]:
# Read the file
file_path = 'batter_historical/682928_22.xlsx'
df = pd.read_excel(file_path)

# Remove (1) or (2) from the dates
df['Date'] = df['Date'].apply(lambda x: re.sub(r'\s\(\d\)', '', str(x)))

# Add year to the date
df['Date'] = pd.to_datetime(df['Date'].astype(str) + ' 2022', errors='coerce').dt.strftime('%Y-%m-%d')

# Add 'dbl' column
dbl_indices = df[df['Date'].str.contains(r'\(\d\)')].index
dbl_values = [re.findall(r'\((\d)\)', date)[0] for date in df.loc[dbl_indices, 'Date']]
df.loc[dbl_indices, 'dbl'] = dbl_values

# Print the modified DataFrame
print(df)

C:\Users\kesse\AppData\Local\Temp\ipykernel_21664\1005030827.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'].astype(str) + ' 2022', errors='coerce').dt.strftime('%Y-%m-%d')


ValueError: Cannot mask with non-boolean array containing NA / NaN values